In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18
from PIL import Image

# Parameters
batch_size = 128
epsilon = 0.2  # Perturbation magnitude for FGSM
train_adv_dir = './train_adv_mnist'
test_adv_dir = './test_adv_mnist'

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transformations for MNIST (grayscale to 3 channels for resnet compatibility)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert 1 channel to 3 channels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Normalize to match model expectations
])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define a CNN model
model = resnet18(weights=None, num_classes=10).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# FGSM Attack function
def fgsm_attack(image, epsilon, data_grad):
    perturbed_image = image + epsilon * data_grad.sign()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# Create directories to save adversarial images
os.makedirs(train_adv_dir, exist_ok=True)
os.makedirs(test_adv_dir, exist_ok=True)

# Function to generate and save adversarial images
def generate_and_save_adversarial_images(loader, epsilon, model, output_dir):
    model.eval()
    for batch_idx, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        images.requires_grad = True

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        model.zero_grad()
        loss.backward()
        
        # Generate adversarial examples
        data_grad = images.grad.data
        perturbed_images = fgsm_attack(images, epsilon, data_grad)

        # Save each image
        for i in range(perturbed_images.size(0)):
            adv_img = perturbed_images[i].detach().cpu()
            adv_img = transforms.ToPILImage()(adv_img)
            label = labels[i].item()
            image_path = os.path.join(output_dir, f"{batch_idx * batch_size + i}_label{label}.png")
            adv_img.save(image_path)

# Train the model (for demonstration, train for a few epochs)
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Generate and save adversarial images for training and testing datasets
print("Generating adversarial training images...")
generate_and_save_adversarial_images(train_loader, epsilon, model, train_adv_dir)

print("Generating adversarial testing images...")
generate_and_save_adversarial_images(test_loader, epsilon, model, test_adv_dir)

print("Adversarial images saved successfully.")


Generating adversarial training images...
Generating adversarial testing images...
Adversarial images saved successfully.
